### <span style="color:blue"> *---------------------------------------*</span>
# <span style="color:blue"> *UDV project*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
# Imports libraries

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import numpy as np

from torchvision import transforms
from torchvision import models
from collections import OrderedDict

import pickle
import time
import matplotlib.pyplot as plt

In [ ]:
# Set the global seed for reproducibility

public_seed = 529
torch.manual_seed(public_seed)
print("Current Seed is {0}".format(public_seed))

In [ ]:
# Obtain device

from udvFunctions.udvDevice import get_device
device = get_device()

In [ ]:
# Load dataset

from udvFunctions.udvClassDataset import MNIST_Pre

In [ ]:
# Prepare UDV constraints

from udvFunctions.udvConstraints import Matrix_bothside, UDV_Diag
from udvFunctions.udvConstraints import Vector_left_U, Vector_right_V

In [ ]:
# Prepare the single-connection layer that carry weight matrix 'w'

from udvFunctions.udvDiagonalLayer import D_singleConnection   

In [ ]:
# Prepare training framework

# Model_0: Matrix_uwv or reluD_uwv
from udvFunctions.udvClassMatrixUWV import udv_frame_m_uwv

# Model_1: Matrix_uv or reluD_uv
from udvFunctions.udvClassMatrixUV import udv_frame_m_uv

# Model_4: ReLU
from udvFunctions.udvClassReLU import udv_frame_relu

# Model_5: Matrix_uv in ReLU
from udvFunctions.udvClassReLU import udvrelu_frame_m_uv


In [ ]:
# Identical initialisation

from udvFunctions.udvSameInit import seedList1

In [ ]:
# Other customised functions

from udvFunctions.udvOtherFunctions import store_metrics, take_avg, check_shapes

### <span style="color:blue"> *---------------------------------------*</span>
# <span style="color:blue"> *Setting Experiments:*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
from udvFunctions.udvClassObtainModels import model_InitPara, revised_model_udvrelu
torch.manual_seed(public_seed)

# Load dataset
data_path = "./MNIST"
train_dataset, val_dataset, num_output = MNIST_Pre(load_All = True, data_path = data_path)

# Set number of epochs and how many different seeds for training 
num_epochs = 70        # Number of epochs; Min:5
num_seeds = 1          # Number of seeds;  Min:1

# Define top layer (classifier)
trans_model = 'efficientnet_b0' # 'efficientnet_b0' or 'maxvit_t' or 'regnet_x_32gf'
num_input, num_hidden_1, batch_size = model_InitPara(name = trans_model, scale_factor = 2/3)

# Is feature layers trainable? (All layers before the classifier)
train_features = True           # Either 'True' (all layers trainable) or 'False' (Only classifier trainable)

# Is pre-trained weights used?
pre_trained = True              # Either False (no pre-trained weights) or True (pre-trained weights from ImageNet-1K)

# Load data
num_workers = 4
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = num_workers)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = num_workers)

# Set optimiser (later) and loss function
optimiser_name = 'Adam'             # 'Adam', 'NAdam', 'SGD', 'SGDM'
SGD_M = 0.9                         # Only available when 'SGDM' is specified in optimiser_name
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()

# Constraints setting
vector_u_norm = 1
vector_v_norm = 1
matrix_uvnorm = 1
d_threshold = 0
d_boundto = 0

# time record 
time_record = []

# Set result path
result_path = './{0}_{1}_{2}_H{3}_BS{4}_E{5}_S{6}_TrainFea{7}_Pre{8}/SingleLayer'.format(trans_model, optimiser_name, learning_rate, num_hidden_1, batch_size, num_epochs, num_seeds, int(train_features), pre_trained)
if not os.path.exists(result_path):
    os.makedirs(result_path)

# Re-producible setting
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with uwv constraints (Matrix): Model_0_UDV*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_ulist1, init_wlist1, init_ulist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_0 = []
full_val_loss_model_0 = []
full_train_acc_model_0 =[]
full_val_acc_model_0 = []

u_1_M_model_0 = []
w_1_M_model_0 = []
u_2_M_model_0 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)
constraints_d = UDV_Diag(threshold = d_threshold, boundTo = d_boundto)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):
    model = revised_model_udvrelu(name = trans_model,
                                  train_features = train_features,
                                  pre_trained = pre_trained, 
                                  model_order = 0, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)
    
    with torch.no_grad():
        check_shapes(model_weight = model.classifier.fc1.weight, list_sample = init_ulist1[i-1])
        check_shapes(model_weight = model.classifier.diag1.weight, list_sample = init_wlist1[i-1])
        check_shapes(model_weight = model.classifier.fc2.weight, list_sample = init_ulist2[i-1])
        model.classifier.fc1.weight = nn.Parameter(init_ulist1[i-1])
        model.classifier.diag1.weight = nn.Parameter(init_wlist1[i-1])
        model.classifier.fc2.weight = nn.Parameter(init_ulist2[i-1])
    
    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, train_accs, val_accs, save_weights_list = udv_frame_m_uwv(model = model, 
                                                                                               optimizer = optimizer, 
                                                                                               loss_fn = loss_fn, 
                                                                                               train_loader = train_dataloader, 
                                                                                               val_loader = val_dataloader, 
                                                                                               num_epochs = num_epochs, 
                                                                                               device = device,
                                                                                               constraints_uv = constraints_uv,
                                                                                               constraints_d = constraints_d,
                                                                                              ) 
    full_train_loss_model_0 = store_metrics(full_train_loss_model_0, train_losses)
    full_val_loss_model_0 = store_metrics(full_val_loss_model_0, val_losses)
    full_train_acc_model_0 = store_metrics(full_train_acc_model_0, train_accs)
    full_val_acc_model_0 = store_metrics(full_val_acc_model_0, val_accs)
    
    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.classifier.diag1.weight, list_sample = save_weights_list[1])
    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = save_weights_list[2])
    u_1_M_model_0.append(save_weights_list[0])
    w_1_M_model_0.append(save_weights_list[1])
    u_2_M_model_0.append(save_weights_list[2])
    
    torch.save(model.state_dict(),'{0}/Seed_{1}_finalEpoch_{2}_model_0.pt'.format(result_path, i, num_epochs))

avg_train_loss_model_0 = take_avg(full_train_loss_model_0)
avg_val_loss_model_0 = take_avg(full_val_loss_model_0)
avg_train_acc_model_0 = take_avg(full_train_acc_model_0)
avg_val_acc_model_0 = take_avg(full_val_acc_model_0)
 
end_time = time.time()
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_ulist1, init_wlist1, init_ulist2, model, constraints_uv, constraints_d, save_weights_list, train_losses, val_losses, train_accs, val_accs

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with uv constraints (Matrix): Model_1_UDV-s*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_ulist1, init_wlist1, init_ulist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_1 = []
full_val_loss_model_1 = []
full_train_acc_model_1 =[]
full_val_acc_model_1 = []

u_1_M_model_1 = []
w_1_M_model_1 = []
u_2_M_model_1 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):  
    model = revised_model_udvrelu(name = trans_model,
                                  train_features = train_features,
                                  pre_trained = pre_trained,
                                  model_order = 1, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)
    
    with torch.no_grad():
        check_shapes(model_weight = model.classifier.fc1.weight, list_sample = init_ulist1[i-1])
        check_shapes(model_weight = model.classifier.diag1.weight, list_sample = init_wlist1[i-1])
        check_shapes(model_weight = model.classifier.fc2.weight, list_sample = init_ulist2[i-1])
        model.classifier.fc1.weight = nn.Parameter(init_ulist1[i-1])
        model.classifier.diag1.weight = nn.Parameter(init_wlist1[i-1])
        model.classifier.fc2.weight = nn.Parameter(init_ulist2[i-1])

    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, train_accs, val_accs, save_weights_list = udv_frame_m_uv(model = model, 
                                                                                              optimizer = optimizer, 
                                                                                              loss_fn = loss_fn, 
                                                                                              train_loader = train_dataloader, 
                                                                                              val_loader = val_dataloader, 
                                                                                              num_epochs = num_epochs, 
                                                                                              device = device,
                                                                                              constraints_uv = constraints_uv,
                                                                                             )
    full_train_loss_model_1 = store_metrics(full_train_loss_model_1, train_losses)
    full_val_loss_model_1 = store_metrics(full_val_loss_model_1, val_losses)
    full_train_acc_model_1 = store_metrics(full_train_acc_model_1, train_accs)
    full_val_acc_model_1 = store_metrics(full_val_acc_model_1, val_accs)
    
    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.classifier.diag1.weight, list_sample = save_weights_list[1])
    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = save_weights_list[2])
    u_1_M_model_1.append(save_weights_list[0])
    w_1_M_model_1.append(save_weights_list[1])
    u_2_M_model_1.append(save_weights_list[2])
    
    torch.save(model.state_dict(),'{0}/Seed_{1}_finalEpoch_{2}_model_1.pt'.format(result_path, i, num_epochs))

avg_train_loss_model_1 = take_avg(full_train_loss_model_1)
avg_val_loss_model_1 = take_avg(full_val_loss_model_1)
avg_train_acc_model_1 = take_avg(full_train_acc_model_1)
avg_val_acc_model_1 = take_avg(full_val_acc_model_1)

end_time = time.time()
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_ulist1, init_wlist1, init_ulist2, model, constraints_uv, save_weights_list, train_losses, val_losses, train_accs, val_accs

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with uwv constraints (ReLUD): Model_2_UDV_ReLU*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_ulist1, init_wlist1, init_ulist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_2 = []
full_val_loss_model_2 = []
full_train_acc_model_2 =[]
full_val_acc_model_2 = []

u_1_M_model_2 = []
w_1_M_model_2 = []
u_2_M_model_2 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)
constraints_d = UDV_Diag(threshold = d_threshold, boundTo = d_boundto)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):
    model = revised_model_udvrelu(name = trans_model,
                                  train_features = train_features,
                                  pre_trained = pre_trained, 
                                  model_order = 2, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)
    
    with torch.no_grad():
        check_shapes(model_weight = model.classifier.fc1.weight, list_sample = init_ulist1[i-1])
        check_shapes(model_weight = model.classifier.diag1.weight, list_sample = init_wlist1[i-1])
        check_shapes(model_weight = model.classifier.fc2.weight, list_sample = init_ulist2[i-1])
        model.classifier.fc1.weight = nn.Parameter(init_ulist1[i-1])
        model.classifier.diag1.weight = nn.Parameter(init_wlist1[i-1])
        model.classifier.fc2.weight = nn.Parameter(init_ulist2[i-1])
    
    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, train_accs, val_accs, save_weights_list = udv_frame_m_uwv(model = model, 
                                                                                               optimizer = optimizer, 
                                                                                               loss_fn = loss_fn, 
                                                                                               train_loader = train_dataloader, 
                                                                                               val_loader = val_dataloader, 
                                                                                               num_epochs = num_epochs, 
                                                                                               device = device,
                                                                                               constraints_uv = constraints_uv,
                                                                                               constraints_d = constraints_d,
                                                                                              ) 
    full_train_loss_model_2 = store_metrics(full_train_loss_model_2, train_losses)
    full_val_loss_model_2 = store_metrics(full_val_loss_model_2, val_losses)
    full_train_acc_model_2 = store_metrics(full_train_acc_model_2, train_accs)
    full_val_acc_model_2 = store_metrics(full_val_acc_model_2, val_accs)
    
    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.classifier.diag1.weight, list_sample = save_weights_list[1])
    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = save_weights_list[2])
    u_1_M_model_2.append(save_weights_list[0])
    w_1_M_model_2.append(save_weights_list[1])
    u_2_M_model_2.append(save_weights_list[2])
    
    torch.save(model.state_dict(),'{0}/Seed_{1}_finalEpoch_{2}_model_2.pt'.format(result_path, i, num_epochs))

avg_train_loss_model_2 = take_avg(full_train_loss_model_2)
avg_val_loss_model_2 = take_avg(full_val_loss_model_2)
avg_train_acc_model_2 = take_avg(full_train_acc_model_2)
avg_val_acc_model_2 = take_avg(full_val_acc_model_2)
 
end_time = time.time()
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_ulist1, init_wlist1, init_ulist2, model, constraints_uv, constraints_d, save_weights_list, train_losses, val_losses, train_accs, val_accs

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with uv constraints (ReLUD): Model_3_UDV_ReLU-s*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_ulist1, init_wlist1, init_ulist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_3 = []
full_val_loss_model_3 = []
full_train_acc_model_3 =[]
full_val_acc_model_3 = []

u_1_M_model_3 = []
w_1_M_model_3 = []
u_2_M_model_3 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):  
    model = revised_model_udvrelu(name = trans_model,
                                  train_features = train_features,
                                  pre_trained = pre_trained, 
                                  model_order = 3, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)
    
    with torch.no_grad():
        check_shapes(model_weight = model.classifier.fc1.weight, list_sample = init_ulist1[i-1])
        check_shapes(model_weight = model.classifier.diag1.weight, list_sample = init_wlist1[i-1])
        check_shapes(model_weight = model.classifier.fc2.weight, list_sample = init_ulist2[i-1])
        model.classifier.fc1.weight = nn.Parameter(init_ulist1[i-1])
        model.classifier.diag1.weight = nn.Parameter(init_wlist1[i-1])
        model.classifier.fc2.weight = nn.Parameter(init_ulist2[i-1])

    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, train_accs, val_accs, save_weights_list = udv_frame_m_uv(model = model, 
                                                                                              optimizer = optimizer, 
                                                                                              loss_fn = loss_fn, 
                                                                                              train_loader = train_dataloader, 
                                                                                              val_loader = val_dataloader, 
                                                                                              num_epochs = num_epochs, 
                                                                                              device = device,
                                                                                              constraints_uv = constraints_uv,
                                                                                             )
    full_train_loss_model_3 = store_metrics(full_train_loss_model_3, train_losses)
    full_val_loss_model_3 = store_metrics(full_val_loss_model_3, val_losses)
    full_train_acc_model_3 = store_metrics(full_train_acc_model_3, train_accs)
    full_val_acc_model_3 = store_metrics(full_val_acc_model_3, val_accs)
    
    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.classifier.diag1.weight, list_sample = save_weights_list[1])
    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = save_weights_list[2])
    u_1_M_model_3.append(save_weights_list[0])
    w_1_M_model_3.append(save_weights_list[1])
    u_2_M_model_3.append(save_weights_list[2])
    
    torch.save(model.state_dict(),'{0}/Seed_{1}_finalEpoch_{2}_model_3.pt'.format(result_path, i, num_epochs))

avg_train_loss_model_3 = take_avg(full_train_loss_model_3)
avg_val_loss_model_3 = take_avg(full_val_loss_model_3)
avg_train_acc_model_3 = take_avg(full_train_acc_model_3)
avg_val_acc_model_3 = take_avg(full_val_acc_model_3)

end_time = time.time()
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_ulist1, init_wlist1, init_ulist2, model, constraints_uv, save_weights_list, train_losses, val_losses, train_accs, val_accs

#### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with ReLU: Model_4_UV_ReLU*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_llist1, init_wlist1, init_llist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )



full_train_loss_model_4 = []
full_val_loss_model_4 = []
full_train_acc_model_4 =[]
full_val_acc_model_4 = []

l_1_model_4 = []
l_2_model_4 = []

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1): 
    model = revised_model_udvrelu(name = trans_model,
                                  train_features = train_features,
                                  pre_trained = pre_trained, 
                                  model_order = 4, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)
    
    with torch.no_grad():
        check_shapes(model_weight = model.classifier.fc1.weight, list_sample = init_llist1[i-1])
        check_shapes(model_weight = model.classifier.fc2.weight, list_sample = init_llist2[i-1])
        model.classifier.fc1.weight = nn.Parameter(init_llist1[i-1])
        model.classifier.fc2.weight = nn.Parameter(init_llist2[i-1])
        
    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, train_accs, val_accs, save_weights_list = udv_frame_relu(model = model, 
                                                                                              optimizer = optimizer, 
                                                                                              loss_fn = loss_fn, 
                                                                                              train_loader = train_dataloader, 
                                                                                              val_loader = val_dataloader, 
                                                                                              num_epochs = num_epochs, 
                                                                                              device = device
                                                                                             )    
    full_train_loss_model_4 = store_metrics(full_train_loss_model_4, train_losses)
    full_val_loss_model_4 = store_metrics(full_val_loss_model_4, val_losses)
    full_train_acc_model_4 = store_metrics(full_train_acc_model_4, train_accs)
    full_val_acc_model_4 = store_metrics(full_val_acc_model_4, val_accs)
    
    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = save_weights_list[1])
    l_1_model_4.append(save_weights_list[0])
    l_2_model_4.append(save_weights_list[1])
    
    torch.save(model.state_dict(),'{0}/Seed_{1}_finalEpoch_{2}_model_4.pt'.format(result_path, i, num_epochs))
    
avg_train_loss_model_4 = take_avg(full_train_loss_model_4)
avg_val_loss_model_4 = take_avg(full_val_loss_model_4)
avg_train_acc_model_4 = take_avg(full_train_acc_model_4)
avg_val_acc_model_4 = take_avg(full_val_acc_model_4)

end_time = time.time()
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_llist1, init_wlist1, init_llist2, model, save_weights_list, train_losses, val_losses, train_accs, val_accs

#### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *The following model with ReLU_uv: Model_5_UV-ReLU(constrained)*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
torch.manual_seed(public_seed)

init_llist1, init_wlist1, init_llist2 = seedList1(num_seeds = num_seeds, 
                                                  public_seed = public_seed,
                                                  num_input = num_input,
                                                  num_hidden_1 = num_hidden_1, 
                                                  num_output = num_output
                                                 )

full_train_loss_model_5 = []
full_val_loss_model_5 = []
full_train_acc_model_5 =[]
full_val_acc_model_5 = []

l_1_model_5 = []
l_2_model_5 = []

constraints_uv = Matrix_bothside(normLim = matrix_uvnorm)

start_time = time.time()
time_record.append(start_time)

for i in range (1, num_seeds + 1):  
    model = revised_model_udvrelu(name = trans_model,
                                  train_features = train_features,
                                  pre_trained = pre_trained, 
                                  model_order = 3, 
                                  num_input = num_input,
                                  num_hidden_1 = num_hidden_1, 
                                  num_output = num_output)
    
    with torch.no_grad():
        check_shapes(model_weight = model.classifier.fc1.weight, list_sample = init_llist1[i-1])
        check_shapes(model_weight = model.classifier.fc2.weight, list_sample = init_llist2[i-1])
        model.classifier.fc1.weight = nn.Parameter(init_llist1[i-1])
        model.classifier.fc2.weight = nn.Parameter(init_llist2[i-1])

    optimizer = optim.Adam(model.parameters(), lr = learning_rate) # Or replace 'Adam' by 'NAdam' or 'SGD'
    #optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = SGD_M) # Only available when 'SGDM' is specified in optimiser_name
    
    model, train_losses, val_losses, train_accs, val_accs, save_weights_list = udvrelu_frame_m_uv(model = model, 
                                                                                                  optimizer = optimizer, 
                                                                                                  loss_fn = loss_fn, 
                                                                                                  train_loader = train_dataloader, 
                                                                                                  val_loader = val_dataloader, 
                                                                                                  num_epochs = num_epochs, 
                                                                                                  device = device,
                                                                                                  constraints_uv = constraints_uv,
                                                                                                 )
    full_train_loss_model_5 = store_metrics(full_train_loss_model_5, train_losses)
    full_val_loss_model_5 = store_metrics(full_val_loss_model_5, val_losses)
    full_train_acc_model_5 = store_metrics(full_train_acc_model_5, train_accs)
    full_val_acc_model_5 = store_metrics(full_val_acc_model_5, val_accs)
    
    check_shapes(model_weight = model.classifier.fc1.weight, list_sample = save_weights_list[0])
    check_shapes(model_weight = model.classifier.fc2.weight, list_sample = save_weights_list[1])
    l_1_model_5.append(save_weights_list[0])
    l_2_model_5.append(save_weights_list[1])
    
    torch.save(model.state_dict(),'{0}/Seed_{1}_finalEpoch_{2}_model_5.pt'.format(result_path, i, num_epochs))

avg_train_loss_model_5 = take_avg(full_train_loss_model_5)
avg_val_loss_model_5 = take_avg(full_val_loss_model_5)
avg_train_acc_model_5 = take_avg(full_train_acc_model_5)
avg_val_acc_model_5 = take_avg(full_val_acc_model_5)

end_time = time.time()
time_record.append(end_time)
duration = end_time - start_time
time_record.append(duration)

del init_llist1, init_wlist1, init_llist2, model, constraints_uv, save_weights_list, train_losses, val_losses, train_accs, val_accs

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *Blank model: Easier to re-use code*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
full_train_loss_model_6 = full_train_loss_model_5
full_val_loss_model_6 = full_val_loss_model_5
full_train_acc_model_6 = full_train_acc_model_5
full_val_acc_model_6 = full_val_acc_model_5

avg_train_loss_model_6 = avg_train_loss_model_5
avg_val_loss_model_6 = avg_val_loss_model_5
avg_train_acc_model_6 = avg_train_acc_model_5
avg_val_acc_model_6 = avg_val_acc_model_5

time_record.append(start_time)
time_record.append(end_time)
time_record.append(duration)

del start_time, end_time, duration

### <span style="color:blue"> *---------------------------------------*</span>
## <span style="color:blue"> *Save Variables*</span>
### <span style="color:blue"> *---------------------------------------*</span>

In [ ]:
# Save variables for analysis

store_file_path = '{0}/results.pkl'.format(result_path)

variables = {
    'num_input': num_input,
    'num_hidden_1': num_hidden_1,
    'num_output': num_output,
    'public_seed': public_seed,
    'data_path': data_path,
    'batch_size': batch_size,
    'num_epochs': num_epochs,
    'num_seeds': num_seeds,
    'learning_rate': learning_rate,
    'device': device,
    'optimiser_name': optimiser_name,
    'trans_model':trans_model,
    
    'full_train_loss_model_0': full_train_loss_model_0,
    'full_train_loss_model_1': full_train_loss_model_1,
    'full_train_loss_model_2': full_train_loss_model_2,
    'full_train_loss_model_3': full_train_loss_model_3,
    'full_train_loss_model_4': full_train_loss_model_4,
    'full_train_loss_model_5': full_train_loss_model_5,
    'full_train_loss_model_6': full_train_loss_model_6,
    'avg_train_loss_model_0': avg_train_loss_model_0,
    'avg_train_loss_model_1': avg_train_loss_model_1,
    'avg_train_loss_model_2': avg_train_loss_model_2,
    'avg_train_loss_model_3': avg_train_loss_model_3,
    'avg_train_loss_model_4': avg_train_loss_model_4,
    'avg_train_loss_model_5': avg_train_loss_model_5,
    'avg_train_loss_model_6': avg_train_loss_model_6,
    'full_val_loss_model_0': full_val_loss_model_0,
    'full_val_loss_model_1': full_val_loss_model_1,
    'full_val_loss_model_2': full_val_loss_model_2,
    'full_val_loss_model_3': full_val_loss_model_3,
    'full_val_loss_model_4': full_val_loss_model_4,
    'full_val_loss_model_5': full_val_loss_model_5,
    'full_val_loss_model_6': full_val_loss_model_6,
    'avg_val_loss_model_0': avg_val_loss_model_0,
    'avg_val_loss_model_1': avg_val_loss_model_1,
    'avg_val_loss_model_2': avg_val_loss_model_2,
    'avg_val_loss_model_3': avg_val_loss_model_3,
    'avg_val_loss_model_4': avg_val_loss_model_4,
    'avg_val_loss_model_5': avg_val_loss_model_5,
    'avg_val_loss_model_6': avg_val_loss_model_6,
    
    'full_train_acc_model_0': full_train_acc_model_0,
    'full_train_acc_model_1': full_train_acc_model_1,
    'full_train_acc_model_2': full_train_acc_model_2,
    'full_train_acc_model_3': full_train_acc_model_3,
    'full_train_acc_model_4': full_train_acc_model_4,
    'full_train_acc_model_5': full_train_acc_model_5,
    'full_train_acc_model_6': full_train_acc_model_6,
    'avg_train_acc_model_0': avg_train_acc_model_0,
    'avg_train_acc_model_1': avg_train_acc_model_1,
    'avg_train_acc_model_2': avg_train_acc_model_2,
    'avg_train_acc_model_3': avg_train_acc_model_3,
    'avg_train_acc_model_4': avg_train_acc_model_4,
    'avg_train_acc_model_5': avg_train_acc_model_5,
    'avg_train_acc_model_6': avg_train_acc_model_6,
    'full_val_acc_model_0': full_val_acc_model_0,
    'full_val_acc_model_1': full_val_acc_model_1,
    'full_val_acc_model_2': full_val_acc_model_2,
    'full_val_acc_model_3': full_val_acc_model_3,
    'full_val_acc_model_4': full_val_acc_model_4,
    'full_val_acc_model_5': full_val_acc_model_5,
    'full_val_acc_model_6': full_val_acc_model_6,
    'avg_val_acc_model_0': avg_val_acc_model_0,
    'avg_val_acc_model_1': avg_val_acc_model_1,
    'avg_val_acc_model_2': avg_val_acc_model_2,
    'avg_val_acc_model_3': avg_val_acc_model_3,
    'avg_val_acc_model_4': avg_val_acc_model_4,
    'avg_val_acc_model_5': avg_val_acc_model_5,
    'avg_val_acc_model_6': avg_val_acc_model_6, 
    
    'u_1_M_model_0': u_1_M_model_0,
    'u_2_M_model_0': u_2_M_model_0,
    'w_1_M_model_0': w_1_M_model_0,
    
    'u_1_M_model_1': u_1_M_model_1,
    'u_2_M_model_1': u_2_M_model_1,
    'w_1_M_model_1': w_1_M_model_1,
    
    'u_1_M_model_2': u_1_M_model_2,
    'u_2_M_model_2': u_2_M_model_2,
    'w_1_M_model_2': w_1_M_model_2,    

    'u_1_M_model_3': u_1_M_model_3,
    'u_2_M_model_3': u_2_M_model_3,
    'w_1_M_model_3': w_1_M_model_3,

    'l_1_model_4': l_1_model_4,
    'l_2_model_4': l_2_model_4,
    
    'l_1_model_5': l_1_model_5,
    'l_2_model_5': l_2_model_5,
    
    'time_record': time_record
}

with open(store_file_path, 'wb') as file:
    pickle.dump(variables, file)

print("Saving pickle file, done")

In [ ]:
from udvFunctions.udvOtherFunctions import avg_stable

last_n = 2            # how many epochs (from end) are averaged (5)
final_train_loss_m0 = avg_stable(avg_train_loss_model_0, last_n)
final_train_loss_m1 = avg_stable(avg_train_loss_model_1, last_n)
final_train_loss_m2 = avg_stable(avg_train_loss_model_2, last_n)
final_train_loss_m3 = avg_stable(avg_train_loss_model_3, last_n)
final_train_loss_m4 = avg_stable(avg_train_loss_model_4, last_n)
final_train_loss_m5 = avg_stable(avg_train_loss_model_5, last_n)
final_train_loss_m6 = avg_stable(avg_train_loss_model_6, last_n)

final_val_loss_m0 = avg_stable(avg_val_loss_model_0, last_n)
final_val_loss_m1 = avg_stable(avg_val_loss_model_1, last_n)
final_val_loss_m2 = avg_stable(avg_val_loss_model_2, last_n)
final_val_loss_m3 = avg_stable(avg_val_loss_model_3, last_n)
final_val_loss_m4 = avg_stable(avg_val_loss_model_4, last_n)
final_val_loss_m5 = avg_stable(avg_val_loss_model_5, last_n)
final_val_loss_m6 = avg_stable(avg_val_loss_model_6, last_n)

final_train_acc_m0 = avg_stable(avg_train_acc_model_0, last_n)
final_train_acc_m1 = avg_stable(avg_train_acc_model_1, last_n)
final_train_acc_m2 = avg_stable(avg_train_acc_model_2, last_n)
final_train_acc_m3 = avg_stable(avg_train_acc_model_3, last_n)
final_train_acc_m4 = avg_stable(avg_train_acc_model_4, last_n)
final_train_acc_m5 = avg_stable(avg_train_acc_model_5, last_n)
final_train_acc_m6 = avg_stable(avg_train_acc_model_6, last_n)

final_val_acc_m0 = avg_stable(avg_val_acc_model_0, last_n)
final_val_acc_m1 = avg_stable(avg_val_acc_model_1, last_n)
final_val_acc_m2 = avg_stable(avg_val_acc_model_2, last_n)
final_val_acc_m3 = avg_stable(avg_val_acc_model_3, last_n)
final_val_acc_m4 = avg_stable(avg_val_acc_model_4, last_n)
final_val_acc_m5 = avg_stable(avg_val_acc_model_5, last_n)
final_val_acc_m6 = avg_stable(avg_val_acc_model_6, last_n)

print('Batch size: {0}\n#Seeds: {1}\n#Epochs: {2}\nOptimier: {3}\nLearning Rate: {4}\n#Hidden_1: {5}'.format(batch_size, num_seeds, num_epochs, optimiser_name, learning_rate, num_hidden_1))
print('Final results are averaged from the last {0} epochs\n'.format(last_n))
print('model_0: UDV                       (avg:-{0}): T-loss {1}; V-loss {2}; T-acc {3}; V-acc {4}'.format(last_n, final_train_loss_m0, final_val_loss_m0, final_train_acc_m0, final_val_acc_m0))
print('model_1: UDV-s                     (avg:-{0}): T-loss {1}; V-loss {2}; T-acc {3}; V-acc {4}'.format(last_n, final_train_loss_m1, final_val_loss_m1, final_train_acc_m1, final_val_acc_m1))
print('model_2: UDV-ReLU                  (avg:-{0}): T-loss {1}; V-loss {2}; T-acc {3}; V-acc {4}'.format(last_n, final_train_loss_m2, final_val_loss_m2, final_train_acc_m2, final_val_acc_m2))
print('model_3: UDV-ReLU-s                (avg:-{0}): T-loss {1}; V-loss {2}; T-acc {3}; V-acc {4}'.format(last_n, final_train_loss_m3, final_val_loss_m3, final_train_acc_m3, final_val_acc_m3))
print('model_4: UV-ReLU                   (avg:-{0}): T-loss {1}; V-loss {2}; T-acc {3}; V-acc {4}'.format(last_n, final_train_loss_m4, final_val_loss_m4, final_train_acc_m4, final_val_acc_m4))
print('model_5: UV-ReLU(constrained)      (avg:-{0}): T-loss {1}; V-loss {2}; T-acc {3}; V-acc {4}'.format(last_n, final_train_loss_m5, final_val_loss_m5, final_train_acc_m5, final_val_acc_m5))